In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [28]:
num_bands = 5
fixed_band_indices = [3]
num_fixed_bands = len(fixed_band_indices)

In [40]:


# num_bands, gridsize1, gridsize2, numtypes
stellarmodel_grid = np.zeros((num_bands, 10, 10, 2))
gridsize1, gridsize2, numtypes = stellarmodel_grid.shape[1:4]

# nobj x num_bands
nobj = 100
obsmags = np.random.randn(nobj, num_bands)
obsmags_var = np.random.randn(nobj, num_bands)**2.0

In [41]:
#TODOS:
# add interpolation and/or prior over smoothness of correction
# turn Gaussian likelihood into student-t or something else
# Priors over stellar parameters!

In [42]:
# gridsize1: logg
# gridsize2: T

# num_bands, gridsize1, gridsize2, numtypes
Stellarmodel_grid = tf.placeholder(shape=[num_bands, gridsize1, gridsize2, numtypes], dtype=tf.float32)

Corrections_dim1 = tf.Variable(np.random.randn(num_bands, gridsize1, numtypes), dtype=tf.float32)
Corrections_dim2 = tf.Variable(np.random.randn(num_bands, gridsize2, numtypes), dtype=tf.float32)

Fixed_band_indices = tf.placeholder(shape=[num_fixed_bands, ], dtype=tf.int32)
tf.scatter_update(Corrections_dim1, Fixed_band_indices, tf.zeros((num_fixed_bands, gridsize1, numtypes)))
tf.scatter_update(Corrections_dim2, Fixed_band_indices, tf.zeros((num_fixed_bands, gridsize1, numtypes)))

# num_bands x gridsize1 x gridsize2 x numtypes
Corrections_grid = Corrections_dim1[:, :, None, :] * Corrections_dim2[:, None, :, :] 

# num_bands, gridsize1, gridsize2, numtypes
Corrected_stellarmodel_grid = Stellarmodel_grid + Corrections_grid

# nobj x num_bands
Obsmags = tf.placeholder(shape=[None, num_bands], dtype=tf.float32)
Obsmags_var = tf.placeholder(shape=[None, num_bands], dtype=tf.float32)

# nobj x num_bands x gridsize1, gridsize2, numtypes
Chi2s = tf.divide(
    tf.pow(Obsmags[:, :, None, None, None] - Corrected_stellarmodel_grid[None, :, :, :, :], 2), 
                  Obsmags_var[:, :, None, None, None])
Logdet = tf.log(Obsmags_var[:, :, None, None, None])

# nobj x gridsize1, gridsize2, numtypes
Loglikes = tf.reduce_sum(-0.5 * Chi2s + Logdet, axis=1)

# nobj
Logevidences = tf.reduce_logsumexp(Loglikes, axis=(1, 2, 3))

# scalar!
MinusLogprob = - tf.reduce_sum(Logevidences, axis=0)

In [46]:
num_iterations = 100
learning_rate = 1e-2

Optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)\
    .minimize(MinusLogprob)
    
with tf.Session() as sess:

    # initialize variables
    sess.run(tf.global_variables_initializer())
    
    # loop: iterations of the optimizer
    for i in range(num_iterations):
        
        subset = np.random.choice(nobj, nobj // 10, replace=False)
        
        _, minuslogprob, corrections_dim1, corrections_dim2 =\
           sess.run([Optimizer, MinusLogprob, Corrections_dim1, Corrections_dim2],
                     feed_dict={
                         Corrected_band_indices: corrected_band_indices,
                         Stellarmodel_grid: stellarmodel_grid,
                         Obsmags: obsmags[subset, :],
                         Obsmags_var: obsmags_var[subset, :],
                     }) 
        
        if i % 1 == 0:
            print(i, minuslogprob)


0 88.46985
1 861.8122
2 804.5259
3 59.14285
4 36.970043
5 651.40155
6 80.64848
7 56.9194
8 117.0749
9 61.545372
10 69.168495
11 51.60427
12 51.878498
13 90.882996
14 73.86728
15 444.14453
16 49.472786
17 67.91606
18 71.17058
19 85.510216
20 63.12753
21 603.19104
22 50.13775
23 37.27345
24 72.032585
25 58.30238
26 107.34381
27 69.61018
28 528.1148
29 64.487076
30 61.251076
31 123.94581
32 320.44214
33 57.876007
34 50.75766
35 84.897354
36 465.51212
37 85.01206
38 61.05448
39 59.405384
40 466.03412
41 470.87592
42 77.50537
43 64.54824
44 85.69429
45 33.00486
46 564.9061
47 47.968143
48 45.028698
49 64.75729
50 406.71222
51 34.266224
52 47.061806
53 32.060852
54 71.54346
55 204.63309
56 107.776
57 49.03289
58 77.82147
59 45.449894
60 79.91129
61 43.943726
62 51.771286
63 85.7027
64 759.63043
65 253.08582
66 68.864334
67 210.4069
68 133.71523
69 71.39722
70 101.12939
71 65.248795
72 44.943245
73 68.45653
74 84.34556
75 509.83792
76 47.499393
77 51.992424
78 378.38364
79 51.080242
80 56.674

In [ ]:
# num_bands, gridsize1, numtypes
corrections_dim1
corrections_dim2